### **CARGA EN BASE DE DATOS: *MONGODB***

MongoDB es un sistema de base de datos NoSQL, orientado a documentos y de código abierto que, en lugar de guardar los datos en tablas, los guarda en estructuras de datos BSON, similar a JSON.

Se cargan en este sistema los cuatro archivos csv generados para futuros análisis de los mismos.

In [2]:

import pandas as pd
from pymongo import MongoClient


In [3]:
llam = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_clean/llamadas_016.csv')
denu = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_clean/denuncias_vg.csv')
vict = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_clean/victimas_mortales_vg.csv')
camp = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_clean/campañas_vg.csv')


In [4]:
client = MongoClient("mongodb://localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [6]:
# Creación de la base de datos:

db = client['impacto_vg']    

In [12]:
# Inserción de los datos:

db['llamadas'].insert_many(llam.to_dict(orient='records'))
db['denuncias'].insert_many(denu.to_dict(orient='records'))
db['victimas'].insert_many(vict.to_dict(orient='records'))
db['campañas'].insert_many(camp.to_dict(orient='records'))

InsertManyResult([ObjectId('6547df605bc8cadf5234ef3a'), ObjectId('6547df605bc8cadf5234ef3b'), ObjectId('6547df605bc8cadf5234ef3c'), ObjectId('6547df605bc8cadf5234ef3d'), ObjectId('6547df605bc8cadf5234ef3e'), ObjectId('6547df605bc8cadf5234ef3f'), ObjectId('6547df605bc8cadf5234ef40'), ObjectId('6547df605bc8cadf5234ef41'), ObjectId('6547df605bc8cadf5234ef42'), ObjectId('6547df605bc8cadf5234ef43')], acknowledged=True)

In [13]:
db.list_collection_names()

['victimas', 'llamadas', 'campañas', 'denuncias']

#### **¿EN QUÉ MES DE 2020 HUBO CAMPAÑA DE PREVENCIÓN?**

In [ ]:

query = { 'Año': 2020}

select = {  'Año': 1,
            '_id': 0,
            'Mes': 1}

resultado = list(db['campañas'].find(query, select))

f'Hubo campaña en {resultado}'


"Hubo campaña en [{'Año': 2020, 'Mes': 11}]"


#### **¿QUÉ PROVINCIAS RECIBIERON MÁS LLAMADAS AL 016 EN LOS MESES SIGUIENTES A LA CAMPAÑA DE NOVIEMBRE 2020?**

In [37]:
query = {'Año': 2020,
         'Mes': {'$in': [11, 12]}}     
select = {  'Provincia': 1,
            '_id': 0,
            'Mes': 1,
            'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).sort('Numero_llamadas', -1).limit(5))

resultado

[{'Provincia': 'Madrid', 'Numero_llamadas': 1087},
 {'Provincia': 'Barcelona', 'Numero_llamadas': 469},
 {'Provincia': 'Valencia/València', 'Numero_llamadas': 322},
 {'Provincia': 'No consta', 'Numero_llamadas': 255},
 {'Provincia': 'Sevilla', 'Numero_llamadas': 213}]

#### **¿Y SI LO COMPARAMOS CON LOS MESES ANTERIORES?**

Ejemplo: Sevilla

In [15]:
query = {'Año': 2020,
         'Mes': {'$in': [8, 9, 10, 11, 12]},
         'Provincia': 'Sevilla'}     
select = { '_id': 0,
            'Mes': 1,
            'Año': 1,
            'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).limit(10))

resultado

[{'Año': 2020, 'Mes': 8, 'Numero_llamadas': 296},
 {'Año': 2020, 'Mes': 9, 'Numero_llamadas': 278},
 {'Año': 2020, 'Mes': 10, 'Numero_llamadas': 207},
 {'Año': 2020, 'Mes': 11, 'Numero_llamadas': 213},
 {'Año': 2020, 'Mes': 12, 'Numero_llamadas': 232}]

In [54]:
query = {'Provincia': {'$in': ['Madrid', 'Barcelona', 'Valencia/València', 'Sevilla']},
         'Año': 2020,
         'Mes': 12}
select = {'Provincia': 1,
          'Mes': 1,
          '_id': 0,
          'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).sort('Numero_llamadas', -1).limit(10))

resultado


[{'Provincia': 'Madrid', 'Mes': 12, 'Numero_llamadas': 1115},
 {'Provincia': 'Barcelona', 'Mes': 12, 'Numero_llamadas': 517},
 {'Provincia': 'Valencia/València', 'Mes': 12, 'Numero_llamadas': 323},
 {'Provincia': 'Sevilla', 'Mes': 12, 'Numero_llamadas': 232}]